In [1]:
import requests

url = "https://developer-apis.awair.is/v1/users/self/devices/awair-element/1714/air-data/latest?fahrenheit=TRUE"
access_token = file1 = open('/Users/matthewkudija/Downloads/awair-access-token.txt','r').read()
payload = {}
headers = {
  'Authorization': 'Bearer '+access_token
}

response = requests.request("GET", url, headers=headers, data = payload)

print(response.text.encode('utf8'))

b'{"data":[{"timestamp":"2020-05-26T14:33:17.010Z","score":83.0,"sensors":[{"comp":"temp","value":81.08600082397462},{"comp":"humid","value":48.63999938964844},{"comp":"co2","value":941.0},{"comp":"voc","value":186.0},{"comp":"pm25","value":3.0}],"indices":[{"comp":"temp","value":2.0},{"comp":"humid","value":0.0},{"comp":"co2","value":1.0},{"comp":"voc","value":0.0},{"comp":"pm25","value":0.0}]}]}'
